In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

def generate_random_number(number,x_data,y_data):
    num_samples = number
    data_numb=np.zeros((num_samples, 28, 100, 1), dtype = np.float32)
    data_labe=np.empty((num_samples, 1), dtype = np.chararray)
    for i in range(num_samples):
        digit1 = np.random.choice(y_data)
        digit2 = np.random.choice(y_data)
        digit3 = np.random.choice(y_data)
        digit4 = np.random.choice(y_data)
        digit5 = np.random.choice(y_data)
        
        length = np.random.randint(1, 6)
        canvas = np.zeros((28, 100, 1), dtype=np.float32)
        labe = np.zeros(length,dtype=np.int_)
        
        lent_front = 0
        for n in range(length):
            digit_labe =[digit1, digit2, digit3, digit4, digit5]
            digit = digit_labe[n]
            id=np.random.choice(np.where(y_train == digit)[0])
            if n==0:
                lent_front = 28*n
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            else:
                numb=np.random.randint(8,18)
                lent_front = lent_front+numb
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            canvas_add = np.concatenate([x_data_front, x_data[id]], 1)
            canvas_add = np.concatenate([canvas_add, x_data_after],  1)
            canvas += canvas_add
            labe[n] = y_data[id]
            
        datalabe = [str(x) for x in labe]
        data_labe[i] = ''.join(datalabe)
        data_numb[i]=canvas 
    print('end')
    return data_numb, data_labe

In [2]:
def get_label(y_lab):
    max_length = max(len(sublist[0]) for sublist in y_lab)
    lab_list = [[int(char) if len(char) == 1 else 11 for char in sublist[0]] + [10] * (max_length - len(sublist[0])) for sublist in y_lab]
    lab = np.array(lab_list)
    return lab

In [3]:
x_data_random , y_data_random = generate_random_number(100000,x_train, y_train)

end


In [4]:
y_data_ran = get_label(y_data_random)

In [5]:
from keras.utils import to_categorical
y_data_ran = to_categorical(y_data_ran, num_classes=11)

In [6]:
x_train_ran , y_train_ran,  y_train_random = x_data_random[:60000] , y_data_ran[:60000]  , y_data_random[:60000]
x_test_ran , y_test_ran,  y_test_random= x_data_random[60000:80000] , y_data_ran[60000:80000] , y_data_random[60000:80000]
x_verify_ran , y_verify_ran, y_verify_random = x_data_random[80000:] , y_data_ran[80000:] , y_data_random[80000:]

In [7]:
from PIL import Image
from IPython.display import display

def image_see(n):
    image_data = (np.squeeze(x_train_ran[n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_train_random[n][0]}"
    print(title)
    display(image)
for id_n in np.random.randint(0, len(y_train_random),10):
    image_see(id_n)



Label:35320


Label:1


Label:52335


Label:0


Label:5


Label:43


Label:07


Label:98


Label:2


Label:487


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau 

input_shape = (28, 100, 1)

#CNN
input_img = Input(shape=input_shape, name="image", dtype="float32")
x = Conv2D(32, (3, 3), activation='tanh')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='tanh')(x)
x = MaxPooling2D((2, 2))(x)
x = Reshape(target_shape=(5, 1472))(x)

#RNN
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.25))(x)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25))(x)

output = Dense(11, activation="softmax")(x) 

model = Model(inputs=input_img, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 28, 100, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 98, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 23, 64)         0         
_________________________________________________________________
reshape (Reshape)            (None, 5, 1472)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 5, 256)           

In [9]:
reduce_lr = ReduceLROnPlateau(patience =3, factor = 0.01, weightdecay = 0.05, verbose = 1) 
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", 
                                               patience=10, restore_best_weights=True)
#train model
history = model.fit(x_train_ran, y_train_ran, epochs=50, verbose=1, batch_size=200, 
          validation_data=(x_verify_ran, y_verify_ran), callbacks=[early_stopping, reduce_lr ])

Epoch 1/50
300/300 [==============================] - 97s 325ms/step - loss: 1.0518 - accuracy: 0.6313 - val_loss: 0.6472 - val_accuracy: 0.7830
Epoch 2/50
300/300 [==============================] - 90s 299ms/step - loss: 0.5627 - accuracy: 0.8128 - val_loss: 0.4303 - val_accuracy: 0.8589
Epoch 3/50
300/300 [==============================] - 85s 283ms/step - loss: 0.4475 - accuracy: 0.8537 - val_loss: 0.3596 - val_accuracy: 0.8842
Epoch 4/50
300/300 [==============================] - 84s 281ms/step - loss: 0.4000 - accuracy: 0.8696 - val_loss: 0.3441 - val_accuracy: 0.8898
Epoch 5/50
300/300 [==============================] - 84s 280ms/step - loss: 0.3734 - accuracy: 0.8788 - val_loss: 0.3202 - val_accuracy: 0.8976
Epoch 6/50
300/300 [==============================] - 83s 277ms/step - loss: 0.3580 - accuracy: 0.8839 - val_loss: 0.3195 - val_accuracy: 0.8984
Epoch 7/50
300/300 [==============================] - 83s 278ms/step - loss: 0.3440 - accuracy: 0.8881 - val_loss: 0.3172 - val_ac

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [11]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test_ran, y_test_ran)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

625/625 [==============================] - 11s 18ms/step - loss: 0.2371 - accuracy: 0.9249
Test Accuracy: 92.49%


In [12]:
y_pred = model.predict(x_test_ran) 
y_pred = np.argmax(y_pred, axis=2)

In [13]:
str_arr = y_pred.astype(str)
str_arr = np.where(str_arr == '10', '', str_arr)
y_test_pred  = [''.join(row) for row in str_arr]
y_test_pred  = np.array(y_test_pred)

In [14]:
prediction = []
for i in range(20000):
    prediction.append(y_test_pred [i])

In [20]:
y_true = []
for i in range(20000):
    y_true.append(y_test_random[i][0])

In [22]:
correct_char = 0
total_char = 0
correct = 0
valid_size=len(y_verify_random)

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

Correct characters predicted : 87.84%
Correct words predicted      : 71.61%


In [23]:
for id_n in np.random.randint(0, len(y_test_random),10):
    image_data = (np.squeeze(x_test_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_test_random[id_n][0]}; Predicted:{y_test_pred[id_n]}"
    print(title)
    display(image)

Label:016; Predicted:016


Label:00036; Predicted:00031


Label:97750; Predicted:97750


Label:4; Predicted:4


Label:222; Predicted:222


Label:30969; Predicted:31968


Label:235; Predicted:235


Label:773; Predicted:773


Label:0020; Predicted:0020


Label:78; Predicted:98


In [52]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

id = list(range(len(x_test)))

np.random.shuffle(id)
id_exam=id[:100]
print(len(id_exam))
for h in id_exam:
    image_data = (np.squeeze(x_test_ran[h]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default() 
    title=f"Lab_{y_test_random[h]}-Pre_{y_test_pred[h]}"
    name=title.replace(" ", "")
    name=name.replace("[", "")
    name=name.replace("]", "")
    file_name = "E:/pythonfile/images-cate/"+name+".png"
    image.save(file_name)

print("Images saved successfully.")

100
Images saved successfully.
